In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import optuna

In [ ]:
# MODEL_TYPE = 'ts_mixer'
MODEL_TYPE = 'tide'
# MODEL_TYPE = 'tft'

RUN_EXP = True
NUM_TRIALS = 100

# for pyfunc model
MODEL_NAME = 'spp_weis'

REMOVE_PRIOR_MODELS = True
TEST_BUILD_BACKTEST = False

In [ ]:
import boto3
import pickle
import random
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
from typing import List, Optional

import requests
from io import StringIO

import ibis
import ibis.selectors as s
from ibis import _
ibis.options.interactive = True

from sklearn.preprocessing import RobustScaler

import torch

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import (
    Scaler,
    MissingValuesFiller,
    Mapper,
    InvertibleMapper,
)
from darts.dataprocessing import Pipeline
from darts.metrics import mape, smape, mae, ope, rmse
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    TFTModel,
    TiDEModel,
    DLinearModel,
    NLinearModel,
    TSMixerModel,
    NaiveEnsembleModel
)


from torchmetrics import (
    SymmetricMeanAbsolutePercentageError, 
    MeanAbsoluteError, 
    MeanSquaredError,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import mlflow

import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:

from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
    plot_pareto_front,
)

In [ ]:
os.chdir('../..')

In [ ]:
# custom modules
import src.data_engineering as de
from src import parameters
from src import plotting
from src.modeling import (
    get_ci_err, build_fit_tsmixerx, build_fit_tide, build_fit_tft, log_pretty
)

## will be loaded from root when deployed
from src.darts_wrapper import DartsGlobalModel

In [ ]:
log.info(f'FORECAST_HORIZON: {parameters.FORECAST_HORIZON}')
log.info(f'INPUT_CHUNK_LENGTH: {parameters.INPUT_CHUNK_LENGTH}')

In [ ]:
torch.set_float32_matmul_precision('medium')

## Data prep

In [ ]:
con = de.create_database()

In [ ]:
con.list_tables()

In [ ]:
con.table('lmp')

In [ ]:
def prep_lmp(
    con: ibis.duckdb.connect,
    start_time: Optional[str] = None,
    end_time: Optional[str] = None,
    loc_filter: str = 'PSCO_',
    clip_outliers: bool = False,
):
    # con = ibis.duckdb.connect("data/spp.ddb", read_only=True)
    lmp = con.table('lmp')
    lmp = lmp.filter(_.Settlement_Location_Name.contains(loc_filter))
    drop_cols = [
        'Interval_HE', 'GMTIntervalEnd_HE', 'timestamp_mst_HE',
        'Settlement_Location_Name', 'PNODE_Name',
        'MLC', 'MCC', 'MEC'
    ]

    if not start_time:
        # get last 1.5 years
        start_time = pd.Timestamp.utcnow() - pd.Timedelta(parameters.TRAIN_START)

    # TODO: handle checks for start_time < end_time
    lmp = lmp.filter(_.timestamp_mst_HE >= start_time)

    
    if clip_outliers:
        clipped_lwr = lmp.LMP.quantile(0.0025)
        clipped_upr = lmp.LMP.quantile(0.9975)
        lmp = (
            lmp
            .mutate(LMP = ibis.ifelse(_.LMP < clipped_upr, _.LMP, clipped_upr))
            .mutate(LMP = ibis.ifelse(_.LMP > clipped_lwr, _.LMP, clipped_lwr))
        )

    if end_time:
        lmp = lmp.filter(_.timestamp_mst_HE <= end_time)

    lmp = (
        lmp
        .mutate(unique_id=_.Settlement_Location_Name)
        .filter(~_.unique_id.contains("_ARPA")) # is missing?
        .drop_null(['unique_id'])
        .mutate(timestamp_mst=_.timestamp_mst_HE)
        .mutate(LMP=_.LMP.cast(parameters.PRECISION))
        .drop(drop_cols)
        .group_by(['unique_id'])
        .order_by(['unique_id', 'timestamp_mst'])
        .mutate(lmp_diff = _.LMP - _.LMP.lag(1))
    )

    for i in [2,3,4,6]:
        win = ibis.window(preceding=i, following=0, group_by=lmp.unique_id, order_by=lmp.timestamp_mst)
        lmp = (
            lmp
            .mutate(lmp.lmp_diff.sum().over(win).cast(parameters.PRECISION).name(f'lmp_diff_rolling_{i}'))
        )
    

    return lmp

In [ ]:
lmp = de.prep_lmp(con)
lmp

In [ ]:
lmp.LMP.min(), lmp.LMP.max()

In [ ]:
lmp_df = lmp.to_pandas().rename(
    columns={
        'LMP': 'LMP_HOURLY',
        'unique_id':'node', 
        'timestamp_mst':'time'
    })

In [ ]:
mtrf = de.prep_mtrf(con)
mtrf

In [ ]:
mtlf = de.prep_mtlf(con)
mtlf

In [ ]:
all_df = de.prep_all_df(con, clip_outliers=True)
all_df

In [ ]:
all_df.LMP.min(), all_df.LMP.max()

In [ ]:
all_df_pd = de.all_df_to_pandas(all_df)
all_df_pd

In [ ]:
all_df_pd.info()

## Prep model training data

In [ ]:
lmp_all, train_all, test_all, train_test_all = de.get_train_test_all(con, clip_outliers=True)

In [ ]:
all_series = de.get_series(lmp_all)
all_series[0].plot()

In [ ]:
train_test_all_series = de.get_series(train_test_all)
train_test_all_series[0].plot()

In [ ]:
train_series = de.get_series(train_all)
train_series[0].plot()

In [ ]:
test_series = de.get_series(test_all)
test_series[0].plot()

In [ ]:
futr_cov = de.get_futr_cov(all_df_pd)
futr_cov[0].plot()

In [ ]:
past_cov = de.get_past_cov(all_df_pd)
past_cov[0].plot()

In [ ]:
con.disconnect()

## Set up hyperparameter tuning study

https://unit8co.github.io/darts/examples/17-hyperparameter-optimization.html?highlight=optuna

In [ ]:
# test build fit function
if TEST_BUILD_BACKTEST:
    
    model = build_fit_tide(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        n_epochs=1)
    
    log.info(f'model.MODEL_TYPE: {model.MODEL_TYPE}')
    
    err_metric = model.backtest(
            series=test_series,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            retrain=False,
            forecast_horizon=parameters.FORECAST_HORIZON,
            stride=25,
            metric=[mae],
            verbose=False,
        )
    
    err_metric = np.mean(err_metric)
    log.info(f'err_metric: {err_metric}')
    
    preds = model.predict(
            series=train_series, 
            n=parameters.FORECAST_HORIZON,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            num_samples=200,
    )
    
    errs = mae(test_series, preds, n_jobs=-1, verbose=True)
    errs = np.mean(errs)
    log.info(f'errs: {errs}')
    
    test_ci_err = get_ci_err(test_series, preds)
    log.info(f'test_ci_err: {test_ci_err}')
    np.mean(test_ci_err)
    log.info(f'np.mean(test_ci_err): {np.mean(test_ci_err)}')
    
    ### test custom metric
    val_backtest = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=parameters.FORECAST_HORIZON,
        stride=25,
        metric=[mae, get_ci_err],
        verbose=False,
        num_samples=200,
        # retrain=False,
    )
    log.info(f'val_backtest: {val_backtest}')
    
    errs = np.mean([e[0] for e in val_backtest])
    ci_err = np.mean([e[1] for e in val_backtest])
    log.info(f'mean backtest errors: errs - {errs} - ci_err {ci_err}')

In [ ]:
def objective_tsmixer(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 32, 256, step=2)
    ff_size = trial.suggest_int("ff_size", 16, 256, step=2)
    num_blocks = trial.suggest_int("num_blocks", 4, 12)
    lr = trial.suggest_float("lr", 1e-5, 1e-4, step=1e-6)
    n_epochs = trial.suggest_int("n_epochs", 4, 12)
    dropout = trial.suggest_float("dropout", 0.4, 0.50, step=0.01)
    activation = trial.suggest_categorical("activation", ['ELU', 'SELU']) # , 'SELU', 'GELU'
    encoder_key = trial.suggest_categorical("encoder_key", ['rel', 'rel_mon', 'rel_mon_day'])
    

    # build and train the tsmixer model with these hyper-parameters:
    model = build_fit_tsmixerx(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        hidden_size=hidden_size,
        ff_size=ff_size,
        num_blocks=num_blocks,
        lr=lr,
        n_epochs=n_epochs,
        dropout=dropout, 
        encoder_key=encoder_key,
        activation=activation,
        callbacks=callback,
        model_id=f"{trial.number:03}",
        log_tensorboard=True,
    )

    model_path = f"{TRIAL_MODEL_DIR}/model_{trial.number}"
    trial.set_user_attr("model_path", model_path)
    model.save(model_path)

    # Evaluate how good it is on the validation set
    val_backtest = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=parameters.FORECAST_HORIZON,
        stride=25,
        metric=[mae, get_ci_err],
        verbose=False,
        num_samples=200,
        # retrain=False,
    )
    
    err_metric = np.mean([e[0] for e in val_backtest])
    ci_error = np.mean([e[1] for e in val_backtest])
    
    if err_metric!= np.nan:
        pass
    else:
        err_metric = float("inf")

    if ci_error!= np.nan:
        pass
    else:
        ci_error = float("inf")
    

    return err_metric , ci_error



In [ ]:
n_futr = futr_cov[0].shape[1]
n_past = past_cov[0].shape[1]

n_futr, n_past

In [ ]:
def objective_tide(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    num_encoder_decoder_layers = trial.suggest_int("num_encoder_decoder_layers", 1, 8)
    decoder_output_dim = trial.suggest_int("decoder_output_dim", 8, 32)
    hidden_size = trial.suggest_int("hidden_size", 8, 64, 1)
    temporal_width_past = trial.suggest_int("temporal_width_past", 0, n_past)
    temporal_width_future = trial.suggest_int("temporal_width_future", 0, n_futr)
    # temporal_width = trial.suggest_int("temporal_width", 0, 2)
    temporal_decoder_hidden  = trial.suggest_int("temporal_decoder_hidden", 4, 64, 1)
    temporal_hidden_size_past = trial.suggest_int("temporal_hidden_size_past", 8, 32, 1)
    temporal_hidden_size_future = trial.suggest_int("temporal_hidden_size_future", 8, 32, 1)
    # temporal_hidden_size = trial.suggest_int("temporal_hidden_size", 4, 16, 1)
    
    lr = trial.suggest_float("lr", 1e-5, 5e-5, step=1e-6)
    n_epochs = trial.suggest_int("n_epochs", 6, 20)
    dropout = trial.suggest_float("dropout", 0.35, 0.5, step=0.01)
    # use_layer_norm = trial.suggest_categorical("use_layer_norm", [True, False])
    # use_reversible_instance_norm = trial.suggest_categorical("use_reversible_instance_norm", [True, False])
    encoder_key = trial.suggest_categorical("encoder_key", ['rel', 'rel_mon', 'rel_mon_day'])
    

    # build and train the tide model with these hyper-parameters:
    model = build_fit_tide(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        num_encoder_decoder_layers=num_encoder_decoder_layers,
        decoder_output_dim=decoder_output_dim,
        hidden_size=hidden_size,
        temporal_width_past=temporal_width_past,
        temporal_width_future=temporal_width_future,
        temporal_decoder_hidden=temporal_decoder_hidden,
        temporal_hidden_size_past=temporal_hidden_size_past,
        temporal_hidden_size_future=temporal_hidden_size_future,
        lr=lr,
        n_epochs=n_epochs,
        dropout=dropout, 
        # use_layer_norm=use_layer_norm,
        # use_reversible_instance_norm=use_reversible_instance_norm,
        encoder_key=encoder_key,
        callbacks=callback,
        model_id=f"{trial.number:03}",
        log_tensorboard=True,
    )

    model_path = f"{TRIAL_MODEL_DIR}/model_{trial.number}"
    trial.set_user_attr("model_path", model_path)
    model.save(model_path)

    # Evaluate how good it is on the validation set
    val_backtest = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=parameters.FORECAST_HORIZON,
        stride=25,
        metric=[mae, get_ci_err],
        verbose=False,
        num_samples=200,
        # retrain=False,
    )
    
    err_metric = np.mean([e[0] for e in val_backtest])
    ci_error = np.mean([e[1] for e in val_backtest])
    
    if err_metric!= np.nan:
        pass
    else:
        err_metric = float("inf")

    if ci_error!= np.nan:
        pass
    else:
        ci_error = float("inf")
    

    return err_metric , ci_error

In [ ]:
def objective_tft(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 8, 32)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 2)
    num_attention_heads = trial.suggest_int("num_attention_heads", 1, 2)
    lr = trial.suggest_float("lr", 1e-4, 1e-3, step=1e-6)
    n_epochs = trial.suggest_int("n_epochs", 2, 6)
    dropout = trial.suggest_float("dropout", 0.3, 0.5, step=0.01)
    full_attention = trial.suggest_categorical("full_attention", [False, True])
    encoder_key = trial.suggest_categorical("encoder_key", ['rel', 'rel_mon', 'rel_mon_day'])
    

    # build and train the tft model with these hyper-parameters:
    model = build_fit_tft(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        lr=lr,
        n_epochs=n_epochs,
        dropout=dropout, 
        encoder_key = encoder_key,
        full_attention=full_attention,
        batch_size=64,
        callbacks=callback,
        model_id=f"{trial.number:03}",
        log_tensorboard=True,
    )

    model_path = f"{TRIAL_MODEL_DIR}/model_{trial.number}"
    trial.set_user_attr("model_path", model_path)
    model.save(model_path)

    # Evaluate how good it is on the validation set
    val_backtest = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=parameters.FORECAST_HORIZON,
        stride=25,
        metric=[mae, get_ci_err],
        verbose=False,
        num_samples=200,
        # retrain=False,
    )
    
    err_metric = np.mean([e[0] for e in val_backtest])
    ci_error = np.mean([e[1] for e in val_backtest])
    
    if err_metric!= np.nan:
        pass
    else:
        err_metric = float("inf")

    if ci_error!= np.nan:
        pass
    else:
        ci_error = float("inf")
    

    return err_metric , ci_error


In [ ]:
os.makedirs(f'study_csv/{MODEL_TYPE}', exist_ok=True)

In [ ]:
def print_callback(study, trial):
    best_smape = min(study.best_trials, key=lambda t: t.values[0])
    best_ci = min(study.best_trials, key=lambda t: t.values[1])
    best_total = min(study.best_trials, key=lambda t: sum(t.values))
    print('\n' + '*'*30, flush=True)
    log.info(f"\nTrial: {trial.number} Current values: {trial.values}")
    log.info(f"Current params: \n{log_pretty(trial.params)}")
    log.info(f"Best {target_names[0]}: Num: {best_smape.number}, {best_smape.values}, Best params: \n{log_pretty(best_smape.params)}")
    log.info(f"Best {target_names[1]}: Num: {best_ci.number}, {best_ci.values}, Best params: \n{log_pretty(best_ci.params)}")
    log.info(f"Best Total: Num: {best_total.number}, {best_total.values}, Best params: \n{log_pretty(best_total.params)}")
    
    study.trials_dataframe().to_csv(f'study_csv/{MODEL_TYPE}/{trial.number:03}.csv')
    

In [ ]:
target_names = ['MAE', 'CI_ERROR']

## Start Experiment

In [ ]:
TRIAL_MODEL_DIR = f'optuna/{MODEL_TYPE}'
MODEL_CHECKPOINT_DIR = f'model_checkpoints/{MODEL_TYPE}_model'

if REMOVE_PRIOR_MODELS:
    try:
        optuna.delete_study(study_name=f"spp_weis_{MODEL_TYPE}", storage="sqlite:///spp_trials.db")
        shutil.rmtree(TRIAL_MODEL_DIR)
        shutil.rmtree(MODEL_CHECKPOINT_DIR)
    except:
        pass
        
os.makedirs(TRIAL_MODEL_DIR, exist_ok=True)
os.makedirs(MODEL_CHECKPOINT_DIR, exist_ok=True)

In [ ]:
if MODEL_TYPE == 'tft':
    objective_func = objective_tft
    
elif MODEL_TYPE == 'tide':
    objective_func = objective_tide
    
elif MODEL_TYPE == 'ts_mixer':
    objective_func = objective_tsmixer

elif MODEL_TYPE == 'dlinear':
    objective_func = objective_dlinear

elif MODEL_TYPE == 'nlinear':
    objective_func = objective_nlinear
    
else:
    raise ValueError(f'Unsuported MODEL_TYPE: {MODEL_TYPE}')

In [ ]:
study_name = f'spp_weis_{MODEL_TYPE}'

In [ ]:
study = optuna.create_study(
        directions=["minimize", "minimize"],
        storage="sqlite:///spp_trials.db", 
        study_name=study_name,
        load_if_exists=True,
    )

In [ ]:
if RUN_EXP:
    study.optimize(objective_func, n_trials=NUM_TRIALS, callbacks=[print_callback])

In [ ]:
for i, target_name in enumerate(target_names):
    fig = plot_optimization_history(study, target=lambda t: t.values[i], target_name=target_name)
    fig.show()

In [ ]:
for i, target_name in enumerate(target_names):
    fig = plot_contour(study, params=["lr", "n_epochs"], target=lambda t: t.values[i], target_name=target_name)
    fig.show()

In [ ]:
plot_param_importances(study)

In [ ]:
plot_pareto_front(study, target_names=target_names)

In [ ]:
plot_pareto_front(study, target_names=target_names, include_dominated_trials=False)

In [ ]:
# TODO: think about how to wieght values to get best model
# best_model = min(study.best_trials, key=lambda t: sum(t.values))
best_model = min(study.best_trials, key=lambda t: t.values[0] + 0.5*t.values[1])
log.info(f"Best number: {best_model.number}")
log.info(f"Best values: {best_model.values}")
log.info(f"Best params: \n{log_pretty(best_model.params)}")

In [ ]:
study.trials_dataframe().to_csv(f'study_csv/test.csv')

In [ ]:
def get_best_trials(
    study_name: str,
    storage: str="sqlite:///spp_trials.db",
    n_results: int=5,
    ci_scaler: float=0.25,
) -> pd.DataFrame:
    study = optuna.load_study(
        study_name=study_name,
        storage=storage, 
    )
    
    trials = pd.DataFrame([{'number': s.number, 'values': s.values, 'params': s.params} for s in study.trials])
    trials['total_value'] = [v[0] + ci_scaler*v[1] if v else np.nan for v in trials['values']]
    trials['model_path'] = [s.user_attrs['model_path'] if s.user_attrs else None for s in study.trials]
    trials = trials[~trials.params.duplicated()]
    best_trials = trials.sort_values('total_value').head(n_results)
    
    
    return best_trials

In [ ]:
best_trials = get_best_trials(study_name, ci_scaler=0.5, n_results=5)
# best_trials['model_path'] = [p.pop('user_attrs_model_path', None) for p in best_trials.params]
best_trials

In [ ]:
# save the list of best params in sxc/parameters.py
[p for p in best_trials.params]

## Create ensemble from best models

In [ ]:
forecasting_models = []
for m in best_trials.model_path:
    if 'ts_mixer' in m.lower():
        forecasting_models += [TSMixerModel.load(m, map_location=torch.device('cpu'))]
        
    elif 'tide' in m.lower():
        forecasting_models += [TiDEModel.load(m, map_location=torch.device('cpu'))]

    elif 'tft' in m.lower():
        forecasting_models += [TFTModel.load(m, map_location=torch.device('cpu'))]

    else:
        raise ValueError(f'Unsuported MODEL_TYPE: {m}')

In [ ]:
# all_models

In [ ]:
loaded_model = NaiveEnsembleModel(
    forecasting_models=forecasting_models, 
    train_forecasting_models=False
)

## Plot test predictions

In [ ]:
plot_ind = 3
plot_series = all_series[plot_ind]

In [ ]:
plot_series.static_covariates.unique_id.LMP

In [ ]:
plot_series.plot()

In [ ]:
plot_end_times = pd.date_range(
    end=test_series[plot_ind].end_time(),
    periods=10,
    freq='d',
)

plot_end_times

In [ ]:
for plot_end_time in plot_end_times:
    log.info(f'plot_end_time: {plot_end_time}')
    
    plot_node_name = plot_series.static_covariates.unique_id.LMP
    
    # if test_end_time < test_series.end_time():
    node_series = plot_series.drop_after(plot_end_time)
        
    log.info(f'plot_end_time: {plot_end_time}')
    log.info(f'node_series.end_time(): {node_series.end_time()}')
    future_cov_series = futr_cov[0]
    past_cov_series = past_cov[0]
    
    data = {
        'series': [node_series.to_json()],
        'past_covariates': [past_cov_series.to_json()],
        'future_covariates': [future_cov_series.to_json()],
        'n': parameters.FORECAST_HORIZON,
        'num_samples': 200
    }
    df = pd.DataFrame(data)
    
    plot_cov_df = future_cov_series.pd_dataframe()
    plot_cov_df = (
        plot_cov_df
        .reset_index()
        .rename(columns={'timestamp_mst':'time', 're_ratio': 'Ratio'})
    )
    
    # Predict on a Pandas DataFrame.
    df['num_samples'] = 500
    
    # for mlflow pyfunc model
    # preds_json = loaded_model.predict(df)
    # preds = TimeSeries.from_json(preds_json)

    # for darts model
    preds = loaded_model.predict(
        series=node_series,
        past_covariates=past_cov_series,
        future_covariates=future_cov_series,
        n=parameters.FORECAST_HORIZON,
        num_samples=500,
    )
    
    q_df = plotting.get_quantile_df(preds, plot_node_name)
    
    plot_df = plotting.get_mean_df(preds, plot_node_name).merge(
        plotting.get_quantile_df(preds, plot_node_name),
        left_index=True,
        right_index=True,
    )
    
    plot_df = plotting.get_plot_df(
            # TimeSeries.from_json(pred),
            preds,
            plot_cov_df,
            lmp_df,
            plot_node_name,
        )
    plot_df.rename(columns={'mean':'mean_fcast'}, inplace=True)
    plot_df
    
    plotting.plotly_forecast(plot_df, plot_node_name, show_fig=True)

## Deprecated MLFlow code

In [ ]:
## MLFlow set up
## mlflow.set_tracking_uri("sqlite:///mlruns.db")
# log.info(f'mlflow.get_tracking_uri(): {mlflow.get_tracking_uri()}')
# exp_name = 'spp_weis'

# if mlflow.get_experiment_by_name(exp_name) is None:
#     exp = mlflow.create_experiment(exp_name)
    
# exp = mlflow.get_experiment_by_name(exp_name)
# exp

In [ ]:
## get signature
# node_series = train_series[0]
# future_cov_series = futr_cov[0]
# past_cov_series = past_cov[0]

# data = {
#     'series': [node_series.to_json()],
#     'past_covariates': [past_cov_series.to_json()],
#     'future_covariates': [future_cov_series.to_json()],
#     'n': parameters.FORECAST_HORIZON,
#     'num_samples': 200
# }

# df = pd.DataFrame(data)

# ouput_example = 'the endpoint return json as a string'

# from mlflow.models import infer_signature
# darts_signature = infer_signature(df, ouput_example)
# darts_signature

In [ ]:
## create and log ensemble model
# with mlflow.start_run(experiment_id=exp.experiment_id) as run:

#     MODEL_TYPE = 'naive_ens'

#     # all_models = models_tsmixer + models_tide + models_tft
#     # fit model with best params from study
#     model = NaiveEnsembleModel(
#         forecasting_models=all_models, 
#         train_forecasting_models=False
#     )

#     model.MODEL_TYPE = MODEL_TYPE
#     model.TRAIN_TIMESTAMP = pd.Timestamp.utcnow()
    
#     log.info(f'run.info: \n{run.info}')
#     artifact_path = "model_artifacts"
    
#     metrics = {}
#     model_params = model.model_params
    
#     # final model back test on validation data
#     acc = model.backtest(
#             series=test_series,
#             past_covariates=past_cov,
#             future_covariates=futr_cov,
#             retrain=False,
#             forecast_horizon=parameters.FORECAST_HORIZON,
#             stride=49,
#             metric=[mae, rmse, get_ci_err],
#             verbose=False,
#             num_samples=200,
#         )

#     mean_acc = np.mean(acc, axis=0)
#     log.info(f'FINAL ACC: mae - {mean_acc[0]} | rmse - {mean_acc[1]} | ci_err - {mean_acc[2]}')
#     acc_df = pd.DataFrame(
#         mean_acc.reshape(1,-1),
#         columns=['mae', 'rmse', 'ci_error']
#     )

#     # add and log metrics
#     metrics['final_mae'] = acc_df.mae[0]
#     metrics['final_rmse'] = acc_df.rmse[0]
#     metrics['final_ci_error'] = acc_df.ci_error[0]
#     mlflow.log_metrics(metrics)

#     # set up path to save model
#     model_path = '/'.join([artifact_path, model.MODEL_TYPE])
#     model_path = '/'.join([artifact_path, 'ens_models'])

#     shutil.rmtree(artifact_path, ignore_errors=True)
#     os.makedirs(model_path)

#     # log params
#     mlflow.log_params(model_params)

#     # save model files (model, model.ckpt) 
#     # and load them to artifacts when logging the model
#     # model.save(model_path)
    
#     for i, m in enumerate(all_models):
#         m.save(f'{model_path}/{m.MODEL_TYPE}_{i}')

#     # save MODEL_TYPE to artifacts
#     # this will be used to load the model from the artifacts
#     model_type_path = '/'.join([artifact_path, 'MODEL_TYPE.pkl'])
#     with open(model_type_path, 'wb') as handle:
#         pickle.dump(model.MODEL_TYPE, handle)

#     model_timestamp = '/'.join([artifact_path, 'TRAIN_TIMESTAMP.pkl'])
#     with open(model_timestamp, 'wb') as handle:
#         pickle.dump(model.TRAIN_TIMESTAMP, handle)
    
#     # map model artififacts in dictionary
#     artifacts = {f:f'{artifact_path}/{f}' for f in os.listdir('model_artifacts')}
#     artifacts['model'] = model_path
    
#     # log model
#     # https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#pip-requirements-example
#     mlflow.pyfunc.log_model(
#         artifact_path='GlobalForecasting',
#         code_path=['src/darts_wrapper.py'],
#         signature=darts_signature,
#         artifacts=artifacts,
#         python_model=DartsGlobalModel(), 
#         pip_requirements=["-r notebooks/model_training/requirements.txt"],
#         registered_model_name=parameters.MODEL_NAME,
#     )


In [ ]:
## Get latest run
# runs = mlflow.search_runs(
#     experiment_ids = exp.experiment_id,
#     # order_by=['metrics.test_mae']
#     order_by=['end_time']
#     )

# runs.sort_values('end_time', ascending=False, inplace=True)
# runs.head()

# best_run_id = runs.run_id.iloc[0]
# best_run_id

# runs['artifact_uri'].iloc[0]

# model_path = runs['artifact_uri'].iloc[0] + '/GlobalForecasting'

# loaded_model = mlflow.pyfunc.load_model(model_path)